In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display
from PIL import Image

import os
# check for set environment variable JB_NOSHOW
show = True
if 'JB_NOSHOW' in os.environ:
    show = False

# Basic FDS Example I

As a basic example we create a FDS simulation setup with a simple gas burner. The gas burner shall have an edge length of $\mf 0.4~m$ and should be raised $\mf 0.2~m$ above the ground. It shall provide $\mf 63~kW$ in a steady-state. The computational domain shall be $\mf 2.0~m$ tall and have an extent of $\mf 2.0~m$ in the x and y axis respectively. The fluid cells shall be cube-shaped with an edge length of $\mf 0.2~m$. The overall simulation time should be $\mf 60~s$.

For details on the pool fires and gas burners look at section Pool Fires.


First you should create a new folder. In this directory the input file is to be saved later. All the generated output files will also be located here. It is useful to create individual directories for each simulation to not accidentially overwrite or mix up files. Create an empty plain text file. You could use any text editor (not a word processor) to create your text file. Although the file extenison does not matter, a common practice is to use the `.fds` extension.

The syntax of FDS input files is given by FORTRAN name lists. These name lists start with an ampersand character and are followed by four capital letters. They end with a slash `/` character. Between these two, parameters of the respective name list can be provided, for example: 

`&NAME P1=value P2=value /`

The first entry in an FDS input file is `&HEAD /` and it ends with `&TAIL /`. In between head and tail of the file the definition of the simulation setup is provided by the user. 


## Meta Data of Simulation Setup (HEAD, TAIL)

Now you will start defining the simulation setup. Two parameters that can be provided to `HEAD` name list are the character ID (`CHID`) and a title (`TITLE`). Both values are strings. The character ID is used to label all the files that are created during the simulation. It can be different from the file name of the FDS input file. The title is a human-readable short description of the setup. The file is closed out with `TAIL`. 

```{tip} 
* `CHID`: is the character ID. This is the string used to tag the output files. 
* `TITLE`: is a descriptive text of your simulation.
```

Following the above, your input file should now contain the following lines:


```
&HEAD CHID  = 'Job_1', 
      TITLE = 'Our first FDS simulation' /

&TAIL /
```

## Bounds of Time and Space (MESH, TIME)

### Time (TIME)

The simulation time is specified in seconds, using the `TIME` name list. The parameter `T_END` determines when the simulation ought to stop. Obviously, this is the time *inside* the simulation and not the time the computer takes to conduct the simulation, which would be referred to as "wall clock time". The start time (`T_BEGIN`) could be adjusted as well, but this is seldom necessary, it is $\mf 0~s$ by default. The input line should now look like this:

`&TIME T_END = 60. /`

Aditionally, the rate at which data is saved can be controlled, from the `DUMP` name list. The parameter `NFRAMES` is used to divide the simulation time, i.e. `T_END` - `T_BEGIN`, into n frames. This is a global control, but different recording functions (`DEVC`, `SLCF`, etc.) can be controlled individually. Please refer to the users guide for detailed information how this can be achieved. To safe a single frame per second we use:

`&DUMP NFRAMES=60 /`

```{tip} 
The grid and geometry can be checked in the following way:
* Add `&TIME T_END = 0/` to your script, this will only build the geometry without actually running a simulation
* Save the FDS input file
* Run the fds input file
* Open the smokeview output file and check the geometry
* To display the grid, press `g`
```


### Space (MESH)
To conduct a simulation we need to  define a computational domain and a time frame. 

The domain is a cuboid shape that is divided in smaller volumes (cells), using the `MESH` name list. It can be given a label (`ID`) to organise larger files with many `MESH`es. The domain volume is defined by providing coordinates of two points of opposing corners, see {numref}`fig-fdstutorial-coordinates`. They define the bounding box (`XB`) of the volume. Their values are to be provided in meters. 

:::{figure-md} fig-fdstutorial-coordinates
<img src="./figs/coordinates.png" width="60%">

XB coordinate definition in fds.
:::

The coordinates of the points are provided as a list of six values. At first the two x-coordinates, followed by the two y- and then the two z-coordinates, like this: `XB = x1,x2, y1,y2, z1,z2,`. In our example case the domain is supposed to be a cube with an edge length of $\mf 2.0~m$. Let's position it around the origin point in the x- and y-directions and start the z-axis at 0. This will lead to: `XB = -1.00,1.00, -1.00,1.00, 0.00,2.00,`.

The size of the fluid cells is indirectly defined, by dividing the domain along each axis. Thus, we need to provide integer values for each axis as the divisor. The parameter is `IJK`, where the "I" denotes the dividion along the x-axis, respectively "J" and "K" denote the y- and z-axis. Thus, getting to fluid cells of an edge length of $\mf 0.2~m$, each domain edge needs to be divided by 10.

Combining all the above, the `MESH` definition should look like this:

```
&MESH ID  = 'Domain',
      IJK = 10,10,10,
      XB  = -1.00,1.00,-1.00,1.00,0.00,2.00 /
```

## Questions

How do you adjust the domain that the cells are cubes with an edge length of $\mf 0.1~m$?